In [1]:
import numpy as np
import csv
import os
import random
import json
from keras.models import load_model
import sys
import time
import math
from queue import PriorityQueue
from numba import jit, cuda

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd


# Parameter
INPUT_PATH = "./CSV/"
INPUT_FBFILE_PATH = "./FB2010-1Hr-150-0.txt"
INPUT_MODEL = "./Coflow_model_select.h5"
INPUT_MINMAX = "./min_max.json"
OUTPUT_CSV = "./P4_RECORD/priority_table_size"
OUTPUT_ACCURACY = "./P4_RECORD/classify_record"
OUTPUT_COMPLETION_TIME = "./P4_RECORD/coflow_completion_time"


with open(INPUT_MINMAX) as file_object:
    min_max = json.load(file_object)
    min_data = np.array(min_max['min_num'])
    max_data = np.array(min_max['max_num'])

MODEL = load_model(INPUT_MODEL)
# COFLOW_NUMBER = 100
# FLOW_NUMBER = 10000 
CONTROLLER_UPDATE_TIME = 30
SKETCH_DEPTH = 3
PACKET_CNT_THRESHOLD = 20
INITIAL_TTL = 10000
INIT_QUEUE_LIMIT = 1048576.0 * 10
JOB_SIZE_MULT = 10.0
NUM_JOB_QUEUES = 10
EGRESS_RATE = 5
MAPPING_THRESHOLD = 50
PRIORITY_ASSIGNMENT_RATE = 10000

# Table Size
PRIORITY_TABLE_SIZE = 4096 
PACKET_CNT_TABLE_SIZE = 512 
# PACKET_CNT_TABLE_SIZE = 512 * 2
# PACKET_CNT_TABLE_SIZE = 512 * 4
# PACKET_CNT_TABLE_SIZE = 512 * 8
# PACKET_CNT_TABLE_SIZE = 512 * 16
# PACKET_CNT_TABLE_SIZE = 512 * 32
# PACKET_CNT_TABLE_SIZE = 512 * 64
FLOW_SIZE_TABLE_SIZE = PACKET_CNT_TABLE_SIZE * 4
COFLOW_TABLE_SIZE = 10

counter = 0
# Data Set
fb_data = {}
fb_coflow_size = {}
fb_coflow_priority = {}
mapping_to_grouping_table = {}
current_global_coflow_size = {} #key=cid, value=size
global_coflow_size = {}
coflow_size_estimation_record = {} #key=counter, value=[real cid, global size, mapping size, [local sizes], [queue sizes]]
priority_result = {}
priority_done = set()

class Switch:
    def __init__(self):
        # Queue
        self.coflow_queue = {} # Coflow_ID(key), [[Flows_List], [Real_Coflow_ID], [inbound queue], [outbound queue]] #v
        self.input_queue = [] #v
        self.output_queue = []
        self.wait_queue = PriorityQueue()

        # Table
        self.priority_table = {} # (Match Table) Flow_ID(key), Priority #v
        self.packet_count_table = [[0 for i in range(PACKET_CNT_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] # (Sketch) Packet_Count #v
        self.flow_size_table = [[0 for i in range(FLOW_SIZE_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] # (Sketch) Packet_Count #v
        self.flow_record_table = {} # (in Controller) Flow_ID(key), Coflow_ID, Priority, Size, TTL, Arrival_Time, Size_m, Finish #v
        self.coflow_priority_table = {} # (in Controller) Coflow_ID(key), Coflow_Size, Priority
        # Other
        self.DNN_counter = 0
        self.DNN_right = 0
        self.sketch_flow_size = {} #v
        self.sketch_cnt_err = 0 #v
        self.sketch_size_err = 0 #v
        self.sketch_mean_err = 0 #v
        self.sketch_counter = 0 #v
        self.priority_table_time = []
        self.priority_table_size = []
        self.packet_collision = [[[] for i in range(PACKET_CNT_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] #v
        self.flow_collision = [[[] for i in range(FLOW_SIZE_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] #v
        self.pkt_collision_counter = 0 #v
        self.flow_collision_counter = 0 #v
        self.coflow_completion = {} # Coflow ID(key), Start Time, Completion Time, Duration Time, Coflow Size, Coflow Priority
        self.classify_accuracy_table = {}
        self.classify_result_table = {}
        self.major_queue = {}
        self.mapping_result = {} #key=timer, value=mapping result->{key=local queue id, value=remote queue id}

2023-07-31 23:00:06.350737: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-31 23:00:07.749966: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-31 23:00:07.750062: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-31 23:00:07.783913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 23:00:07.784587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2023-07-31 23:00:07.784626: I tensorflow/stream_executor/cuda/cu

In [2]:
def switch_reset(switch):
    switch.coflow_queue = {} # Coflow_ID(key), [[Flows_List], [Real_Coflow_ID], [inbound queue], outbound queue] #v
    #switch.input_queue = [] #v
    switch.output_queue = []
    switch.wait_queue = PriorityQueue()

    # Table
    switch.priority_table = {} # (Match Table) Flow_ID(key), Priority #v
    switch.packet_count_table = [[0 for i in range(PACKET_CNT_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] # (Sketch) Packet_Count #v
    switch.flow_size_table = [[0 for i in range(FLOW_SIZE_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] # (Sketch) Packet_Count #v
    switch.flow_record_table = {} # (in Controller) Flow_ID(key), Coflow_ID, Priority, Size, TTL, Arrival_Time, Size_m, Finish #v
    switch.coflow_priority_table = {} # (in Controller) Coflow_ID(key), Coflow_Size, Priority
    # Other
    switch.DNN_counter = 0
    switch.DNN_right = 0
    switch.sketch_flow_size = {} #v
    switch.sketch_cnt_err = 0 #v
    switch.sketch_size_err = 0 #v
    switch.sketch_mean_err = 0 #v
    switch.sketch_counter = 0 #v
    switch.priority_table_time = []
    switch.priority_table_size = []
    switch.packet_collision = [[[] for i in range(PACKET_CNT_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] #v
    switch.flow_collision = [[[] for i in range(FLOW_SIZE_TABLE_SIZE)] for j in range(SKETCH_DEPTH)] #v
    switch.pkt_collision_counter = 0 #v
    switch.flow_collision_counter = 0 #v
    switch.coflow_completion = {} # Coflow ID(key), Start Time, Completion Time, Duration Time, Coflow Size, Coflow Priority
    switch.classify_accuracy_table = {}
    switch.classify_result_table = {}
    switch.major_queue = {}
    switch.mapping_result = {} #key=timer, value=mapping result->{key=local queue id, value=remote queue id}

def global_reset():
    mapping_to_grouping_table.clear()
    current_global_coflow_size.clear()
    #global_coflow_size.clear()
    coflow_size_estimation_record.clear()
    priority_result.clear()
    priority_done.clear()

def readDataSet():
    global fb_data, fb_coflow_size, fb_coflow_priority
    
    def getPriority(size):
        tmp = INIT_QUEUE_LIMIT
        p = 0
        while size > tmp:
            p += 1
            tmp *= JOB_SIZE_MULT
            if p >= NUM_JOB_QUEUES:
                break
        return p

    with open(INPUT_FBFILE_PATH, "r") as f:
        first = True
        for line in f:
            if first == True:
                if first:
                    first = False
                    continue
            line = line.replace('\n', '').split(' ')
            coflow = float(line[0])
            mapper_list = []
            reducer_list = []
            size_list = []
            mapper_num = int(line[2])
            for m in range(mapper_num):
                mapper_list.append(float(line[3+m]))
            reducer_num = int(line[2+int(line[2])+1])
            for r in range(reducer_num):
                reducer_list.append(float(line[2+int(line[2])+1+r+1].split(':')[0]))
                size_list.append(float(line[2+int(line[2])+1+r+1].split(':')[1])) # MB
            fb_coflow_size[str(coflow)] = sum(size_list) * 1024 * 1024
            fb_coflow_priority[str(coflow)] = getPriority(fb_coflow_size[str(coflow)])
            for m in range(mapper_num):
                for r in range(reducer_num):
                    key = str(coflow) + "-" + str(mapper_list[m]) + "-" + str(reducer_list[r])
                    fb_data[key] = size_list[r] / mapper_num * 1024 ###
    # print(fb_data)
    # print(fb_coflow_priority)
    # print(fb_coflow_size)

def loadCsvData():
    def sortDir(s):
        return int(s.split("_")[0])
    input_data = []
    input_data_flow = {}
    f_cnt = 0
    c_list = []
    csv_dir = sorted(os.listdir(INPUT_PATH), key=sortDir) # Sort
    for f1 in csv_dir: # Packet dir
        print("open ", f1)
        for f2 in sorted(os.listdir(os.path.join(INPUT_PATH, f1))): # Host file
            print(f2, end=" ")
            data = np.loadtxt(os.path.join(os.path.join(INPUT_PATH, f1), f2), dtype=float, delimiter=",", skiprows=1, usecols=range(8))
            for i in range(len(data)): # Packets
                c_id = data[i][0] #coflow id
                m_id = data[i][3] #mapper id
                r_id = data[i][4] #reducer id
                key = str(c_id) + "-" + str(m_id) + "-" + str(r_id)
                if c_id not in c_list:
                    #if len(c_list) >= COFLOW_NUMBER:
                    #    continue
                    c_list.append(c_id)
                if key not in input_data_flow.keys():
                    if key not in fb_data.keys() or data[i][7] == 0:
                        continue
                    # if f_cnt >= FLOW_NUMBER:
                    #     continue
                    f_cnt += 1
                    input_data_flow[key] = []
                input_data_flow[key].append(data[i])
            for key in input_data_flow.keys():
                num = (fb_data[key] / input_data_flow[key][0][7])
                if len(input_data_flow[key]) < num:
                    orgin_len = len(input_data_flow[key])
                    while len(input_data_flow[key]) < num:
                        tmp_input = input_data_flow[key].copy()
                        for d in tmp_input:
                            if len(input_data_flow[key]) < num:
                                input_data_flow[key].append(d)
                            else:
                                break
                    add = 0
                    inter = random.sample([1, 2, 3, 4, 6, 7, 8], 1)[0]
                    for i in range(len(input_data_flow[key])-orgin_len):
                        if (orgin_len + i) % inter == 0:
                            add += 1
                        input_data_flow[key][orgin_len + i][5] += add
        print("")
        # if f_cnt >= FLOW_NUMBER:
        #     break
        # if len(c_list) >= COFLOW_NUMBER:
        #    break
    for key in input_data_flow.keys():
        for d in input_data_flow[key]:
            input_data.append(d)
    input_data = sorted(input_data, key=lambda s:s[5])
    f_id_list = input_data_flow.keys()
    return input_data, input_data_flow, f_id_list, c_list

def sampling(input_queue, input_data_flow, f_id_list, c_list, k):
    shuffle_c_list = sorted(c_list).copy()
    random.shuffle(shuffle_c_list)
    sample_c_list = shuffle_c_list[:k]
    set_sample_c_list = set(sample_c_list)
    sample_f_id_list = []
    for fid in f_id_list:
        if float(fid.split('-')[0]) in set_sample_c_list:
            sample_f_id_list.append(fid)
    set_sample_fid_list = set(sample_f_id_list)
    sample_input_data_flow = {}
    for key in input_data_flow:
        if key in set_sample_fid_list:
            sample_input_data_flow[key] = input_data_flow[key]
    sample_input_queue = []
    global_coflow_size.clear()
    for item in input_queue:
        fid = str(item[0]) + "-" + str(item[3]) + "-" + str(item[4])
        size = item[6]
        if item[0] not in global_coflow_size.keys():
            global_coflow_size[item[0]] = size
        else:
            global_coflow_size[item[0]] += size
        if fid in set_sample_fid_list:
            sample_input_queue.append(item)
    return sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list

def grouping(switches, sample_input_queue, sample_input_flow, sample_f_id_list, k):
    shuffle_fid = sample_f_id_list
    random.shuffle(shuffle_fid)
    shuffle_fid_list = np.array_split(shuffle_fid, k)
    shuffle_fid_list_sets = []
    for fid_list in shuffle_fid_list:
        shuffle_fid_list_sets.append(set(fid_list))
    switch_datas = [[]for i in range(k)]
    for item in sample_input_queue:
        key = str(item[0]) + "-" + str(item[3]) + "-" + str(item[4])
        for fid_list_set in shuffle_fid_list_sets:
            if key in fid_list_set:
                switch_datas[shuffle_fid_list_sets.index(fid_list_set)].append(item)
    for switch in switches:
        switch.input_queue = switch_datas[switches.index(switch)]
    return switches

def grouping2(switches, sample_input_queue, sample_input_flow, sample_f_id_list, sample_c_list, numOfSwitches):
    shuffle_cid = sample_c_list
    print("before shuffle: ",shuffle_cid)
    random.shuffle(shuffle_cid)
    print("after shuffle: ", shuffle_cid)
    shuffle_cid_list = np.array_split(shuffle_cid, numOfSwitches)

    shuffle_cid_list_sets = []
    for cid_list in shuffle_cid_list:
        shuffle_cid_list_sets.append(set(cid_list))
        
    switch_datas = [[]for i in range(numOfSwitches)]
    for item in sample_input_queue:
        cid = item[0]
        for cid_list_set in shuffle_cid_list_sets:
            if cid in cid_list_set:
                switch_datas[shuffle_cid_list_sets.index(cid_list_set)].append(item)
    for switch in switches:
        switch.input_queue = switch_datas[switches.index(switch)]
    return switches, shuffle_cid_list

def fair_grouping(switches, sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list, k):
    cidTofid = {}
    for cid in sample_c_list:
        cidTofid[cid]=[]
    for fid in sample_f_id_list:
        tmp = fid.split('-')
        cidTofid[float(tmp[0])].append(fid)
    for key in cidTofid.keys():
        print(key, ":", cidTofid[key])
    fidToswitch = {}
    for cid in cidTofid:
        random.shuffle(cidTofid[cid])
        newarr = np.array_split(cidTofid[cid], k)
        newlist = []
        for arr in newarr:
            newlist.append(list(arr))
        for lis in newlist:
            for fid in lis:
                fidToswitch[fid]=newlist.index(lis)
    final_input_queue = []
    for i in range(k):
        final_input_queue.append([])
    '''
    for key in sample_input_data_flow:
        final_input_queue[fidToswitch[key]].append(sample_input_data_flow[key])
    '''
    for data in sample_input_queue:
        fid = str(data[0]) + "-" + str(data[3]) + "-" + str(data[4])
        final_input_queue[fidToswitch[fid]].append(data)
    for switch in switches:
        switch.input_queue = final_input_queue[switches.index(switch)].copy()
    return switches
    
def getFlowID(packet, f_id_list):
    c_id = packet[0]
    m_id = packet[3]
    r_id = packet[4]
    key = str(c_id) + "-" + str(m_id) + "-" + str(r_id)
    return list(f_id_list).index(key)

def checkPriorityTable(switch, f_id, packet):
    # priority table
    # (Match Table) Flow_ID(key), Priority
    find = False
    if f_id in switch.priority_table.keys():
        packet.append(switch.priority_table[f_id]) # Add priority
        find = True
    else:
        packet.append(0) # Add highest priority
    return find, packet

def hash(key, width, depth):
    h = (key+(depth+1)**(depth)) % width
    return h

def sketchAction(switch, f_id, table, add_value, clear=False):
    global packet_collision, flow_collision, pkt_collision_counter, flow_collision_counter
    get_value = []
    for i in range(SKETCH_DEPTH):
        key = hash(f_id, len(table[i]), i)
        table[i][key] += add_value
        get_value.append(table[i][key])
        if clear:
            table[i][key] = 0
        # ------ Record ------
        if table == switch.packet_count_table: # Add packet count
            if add_value != 0:
                if f_id not in switch.packet_collision[i][key]:
                    if switch.packet_collision[i][key] != []:
                        switch.pkt_collision_counter += 1
                        #print("Packet Count Collision - table ", i, ": ", f_id, " and ", switch.packet_collision[i][key])
                    switch.packet_collision[i][key].append(f_id)
                    #print("put fid into packet collision table[",i,"][",key,"]")
            if clear == True:
                # print(packet_collision[i][key])
                if f_id in switch.packet_collision[i][key]:
                    switch.packet_collision[i][key].remove(f_id)
                # print("Clear key in packte size: ", f_id)
        elif table == switch.flow_size_table: # Add flow size
            if add_value != 0:
                if f_id not in switch.flow_collision[i][key]:
                    if switch.flow_collision[i][key] != []:
                        switch.flow_collision_counter += 1
                        #print("Flow Size Collision - table ", i, ": ", f_id, " and ", switch.flow_collision[i][key])
                    switch.flow_collision[i][key].append(f_id) 
                    #print("put fid into flow collision table[",i,"][",key,"]")
            if clear == True:
                # print(flow_collision[i][key])
                if f_id in switch.flow_collision[i][key]:
                    switch.flow_collision[i][key].remove(f_id)
                # print("Clear key in flow size: ", f_id)
        # ------ Record ------
    return min(get_value)

def updatePacketCntTable(switch, f_id, packet):
    cnt = sketchAction(switch, f_id, switch.packet_count_table, 1, False)
    if cnt == 1 or cnt == PACKET_CNT_THRESHOLD: 
        return True
    else:
        return False
    
def updateFlowSizeTable(switch, f_id, packet):
    size = sketchAction(switch, f_id, switch.flow_size_table, packet[6], False)
    # Record
    if f_id not in switch.sketch_flow_size.keys():
        switch.sketch_flow_size[f_id] = []
    if len(switch.sketch_flow_size[f_id]) < PACKET_CNT_THRESHOLD:
        switch.sketch_flow_size[f_id].append(packet[6])
    # Record
    return size

# function optimized to run on gpu 
#@jit(target_backend='cuda') 
def classify(switch, f_id, real_cid, packet_m, arrival_t):
    def normalize(switch, f_id2, packet_m, arrival_t):
        feature_time = abs(arrival_t - switch.flow_record_table[f_id2][4]) / (max_data[0]-min_data[0])
        normalize_packet1 = (packet_m - min_data[1]) / (max_data[1] - min_data[1])
        normalize_packet2 = (switch.flow_record_table[f_id2][5] - min_data[1]) / (max_data[1] - min_data[1])
        return np.array([[feature_time, normalize_packet1, normalize_packet2]])
    if len(switch.coflow_queue.keys()) == 0: # Create a new queue
        return real_cid # Real coflow ID
    sameScore = []
    diffScore = []
    sorted_coflow_keys = sorted(switch.coflow_queue.keys())
    for i in range(len(sorted_coflow_keys)):
        sameScore.append(0)
        diffScore.append(0)
        cnt = 0
        sampleNum = min(len(switch.coflow_queue[sorted_coflow_keys[i]][0]), 20) #min(num of flows in queue, 20)
        sampleList = random.sample(range(len(switch.coflow_queue[sorted_coflow_keys[i]][0])), sampleNum)
        for j in sampleList: # Each flow in coflow
            if switch.coflow_queue[sorted_coflow_keys[i]][0][j] not in switch.flow_record_table.keys():
                continue
            n = normalize(switch, switch.coflow_queue[sorted_coflow_keys[i]][0][j], packet_m, arrival_t)
            predict_prob = MODEL.predict(n)
            predict_classes = predict_prob[0]
            sameScore[i] += predict_classes[1]
            diffScore[i] += predict_classes[0]
            cnt += 1
            # ------ Record ------
            switch.DNN_counter += 1
            if real_cid == switch.coflow_queue[sorted_coflow_keys[i]][1][j] and predict_classes[1] > predict_classes[0]:
                switch.DNN_right += 1
            if real_cid != switch.coflow_queue[sorted_coflow_keys[i]][1][j] and predict_classes[1] <= predict_classes[0]:
                switch.DNN_right += 1
            # ------ Record ------
        if cnt > 0:
            sameScore[i] /= cnt
            diffScore[i] /= cnt
    score = [] # [c_id, Max Score]
    score.append(-1) #c_id
    score.append(-1) #Max Score
    all_score = {}
    for key in sorted_coflow_keys:
        all_score[key] = 0
    for i in range(len(sorted_coflow_keys)):
        if sameScore[i] > diffScore[i]:
            all_score[sorted_coflow_keys[i]]=sameScore[i]
            if sameScore[i] > score[1]:
                score[1] = sameScore[i]
                score[0] = sorted_coflow_keys[i]
    if score[1] == -1: # No friend and create a new job
        if len(switch.coflow_queue.keys()) < COFLOW_TABLE_SIZE:
            c_id = real_cid # Real coflow ID
        else:
            # Find the smallest coflow
            small = [0, sys.maxsize] # [c_id, #]
            for i in range(len(sorted_coflow_keys)):
                if len(switch.coflow_queue[sorted_coflow_keys[i]][0]) < small[1]:
                    small[0] = sorted_coflow_keys[i]
                    small[1] = len(switch.coflow_queue[sorted_coflow_keys[i]][0])
            c_id = small[0] # Smallest coflow ID
    else:
        c_id = score[0] # Existing coflow ID

    return c_id

# function optimized to run on gpu 
#@jit(target_backend='cuda') 
def classify_for_mapping(switch, packet_m, arrival_t):
    def normalize(switch, f_id2, packet_m, arrival_t):
        feature_time = abs(arrival_t - switch.flow_record_table[f_id2][4]) / (max_data[0]-min_data[0])
        normalize_packet1 = (packet_m - min_data[1]) / (max_data[1] - min_data[1])
        normalize_packet2 = (switch.flow_record_table[f_id2][5] - min_data[1]) / (max_data[1] - min_data[1])
        return np.array([[feature_time, normalize_packet1, normalize_packet2]])
    if len(switch.coflow_queue.keys()) == 0: # Create a new queue
        return -1 # No queue available
    sameScore = []
    diffScore = []
    sorted_coflow_keys = sorted(switch.coflow_queue.keys())
    for i in range(len(sorted_coflow_keys)):
        sameScore.append(0)
        diffScore.append(0)
        cnt = 0
        sampleNum = min(len(switch.coflow_queue[sorted_coflow_keys[i]][0]), 20) #min(num of flows in queue, 20)
        sampleList = random.sample(range(len(switch.coflow_queue[sorted_coflow_keys[i]][0])), sampleNum)
        for j in sampleList: # Each flow in coflow
            if switch.coflow_queue[sorted_coflow_keys[i]][0][j] not in switch.flow_record_table.keys():
                continue
            n = normalize(switch, switch.coflow_queue[sorted_coflow_keys[i]][0][j], packet_m, arrival_t)
            predict_prob = MODEL.predict(n)
            predict_classes = predict_prob[0]
            sameScore[i] += predict_classes[1]
            diffScore[i] += predict_classes[0]
            cnt += 1
        if cnt > 0:
            sameScore[i] /= cnt
            diffScore[i] /= cnt
    score = [] # [c_id, Max Score]
    score.append(-1) #c_id
    score.append(-1) #Max Score
    all_score = {}
    for key in sorted_coflow_keys:
        all_score[key] = 0
    for i in range(len(sorted_coflow_keys)):
        if sameScore[i] > diffScore[i]:
            all_score[sorted_coflow_keys[i]]=sameScore[i]
            if sameScore[i] > score[1]:
                score[1] = sameScore[i]
                score[0] = sorted_coflow_keys[i]
    if score[1] == -1: # No friend and create a new job
        if len(switch.coflow_queue.keys()) < COFLOW_TABLE_SIZE:
            c_id = -1 # no friend
        else:
            # Find the smallest coflow
            small = [0, sys.maxsize] # [c_id, #]
            for i in range(len(sorted_coflow_keys)):
                if len(switch.coflow_queue[sorted_coflow_keys[i]][0]) < small[1]:
                    small[0] = sorted_coflow_keys[i]
                    small[1] = len(switch.coflow_queue[sorted_coflow_keys[i]][0])
            c_id = small[0] # Smallest coflow ID
    else:
        c_id = score[0] # Existing coflow ID

    return c_id

def groundtruth(switch, real_cid):
    result = {}
    sorted_coflow_keys = sorted(switch.coflow_queue.keys())
    for key in sorted_coflow_keys:
        result[key] = 0
    for key in sorted_coflow_keys:
        for cid in switch.coflow_queue[key][1]:
            if cid == real_cid:
                result[key]+=1
    return result

#label manually with some error probability
def label(packet,c_list,percentage):
    prob = random.randrange(0,100)
    if prob<percentage:
        return packet[0]
    else:
        cid = random.choice(c_list)
        while True:
            if cid != packet[0]:
                break
            else:
                cid = random.choice(c_list)
        return cid

def updateFlowRecordTable(switches, switch, f_id, c_list, packet):
    #flow_record_table
    #                                 0          1        2    3         4         5       6
    #(in Controller) Flow_ID(key), Coflow_ID, Priority, Size, TTL, Arrival_Time, Size_m, Finish
    # Get data from Packet Table
    cnt = sketchAction(switch, f_id, switch.packet_count_table, 0, False)
    size = sketchAction(switch, f_id, switch.flow_size_table, 0, False)
    if cnt == 1:
        #print("Put ", f_id, "in Flow Table")
        if f_id in switch.flow_record_table.keys():
            #print("(cnt = 1) Flow ", f_id, " is in Flow Table")
            switch.flow_record_table[f_id][6] = False #finish = false
            switch.flow_record_table[f_id][3] = INITIAL_TTL
        else:
            switch.flow_record_table[f_id] = [None, 0, size, INITIAL_TTL, packet[5], 0, False]
        return
    elif cnt == PACKET_CNT_THRESHOLD:
        sketchAction(switch, f_id, switch.packet_count_table, 0, True) # Reset
        # Classify
        packet_m = size / cnt
        # ------ Record ------
        if f_id in switch.sketch_flow_size.keys(): 
            real_packet_s = sum(switch.sketch_flow_size[f_id])
            real_packet_c = len(switch.sketch_flow_size[f_id])
            real_packet_m = real_packet_s/real_packet_c
            if math.isnan(abs(real_packet_s - size) / real_packet_s) == False and math.isnan(abs(real_packet_c - cnt) / real_packet_c) == False and math.isnan(abs(real_packet_m - packet_m) / (real_packet_s/real_packet_c)) == False:
                switch.sketch_size_err += abs(real_packet_s - size) / real_packet_s
                switch.sketch_cnt_err += abs(real_packet_c - cnt) / real_packet_c
                switch.sketch_mean_err += abs(real_packet_m - packet_m) / (real_packet_s/real_packet_c)
                switch.sketch_counter += 1
                '''
                print("------ ", f_id, " ------")
                print("sketch size: ", size, " real size: ", real_packet_s)
                print("sketch cnt: ", cnt, " real cnt: ", real_packet_c)
                print("sketch mean: ", packet_m, " real mean: ", real_packet_s/real_packet_c)
                print("-----------------")
                '''
        # ------ Record ------
        if f_id not in switch.flow_record_table.keys():
            print("(cnt = ", PACKET_CNT_THRESHOLD, ") Flow ", f_id, " is not in Flow Table")
            switch.flow_record_table[f_id] = [None, 0, size, INITIAL_TTL, packet[5], packet_m, False]
        else:
            switch.flow_record_table[f_id][2] = size
            switch.flow_record_table[f_id][5] = packet_m
        arrival_t = switch.flow_record_table[f_id][4]
        c_id = classify(switch, f_id, packet[0], packet_m, arrival_t)
        
        
        # ------ Record ------
        real_coflow_id = packet[0]
        real_size = fb_coflow_size[str(real_coflow_id)]
        classified_size = fb_coflow_size[str(float(c_id))]
        real_priority = fb_coflow_priority[str(real_coflow_id)]
        classified_priority = fb_coflow_priority[str(float(c_id))]
        with open(OUTPUT_ACCURACY, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([f_id, real_coflow_id, c_id, real_size, classified_size, real_priority, classified_priority])
        # ------ Record ------
        # Update Coflow Data and set priority
        priority = 0
        if c_id in switch.coflow_queue.keys(): # Record in Coflow Queue for classify
            switch.coflow_queue[c_id][0].append(f_id)
            switch.coflow_queue[c_id][1].append(packet[0]) # Real coflow id of this flow 
        else:
            switch.coflow_queue[c_id] = [[f_id],[packet[0]],[],[]] #new queue
        if c_id in switch.coflow_priority_table.keys(): # Update priority
            priority = switch.coflow_priority_table[c_id][1]
        else: # New coflow
            switch.coflow_priority_table[c_id] = [0, 0] #size, priority
        # Update Flow Table
        switch.flow_record_table[f_id][0] = c_id
        switch.flow_record_table[f_id][1] = priority
        # Insert to Priority Table
        if len(switch.priority_table) < PRIORITY_TABLE_SIZE:
            switch.priority_table[f_id] = priority
        else:
            print("(Priority Table) Overflow")
            # Todo
        
        #Method 1: Map major and minor queues
        '''
        if len(switch.coflow_queue[c_id][0])%MAPPING_THRESHOLD==0:
            for remote_switch in switches:
                if remote_switch == switch:
                    continue
                remote_cid = single_queue_mapping(switch, remote_switch, c_id)
                if remote_cid == -1:
                    continue
                #record local queue id in remote switch queue
                remote_switch.coflow_queue[remote_cid][2].append(str(switches.index(switch))+"-"+str(c_id))
                #record remote switch queue id in local switch queue
                outbound_qid = str(switches.index(remote_switch))+"-"+str(remote_cid)
                #first record
                if switch.coflow_queue[c_id][3] == -1:
                    switch.coflow_queue[c_id][3] = outbound_qid
                #outbound redirected
                elif outbound_qid != switch.coflow_queue[c_id][3]:
                    tmp = switch.coflow_queue[c_id][3].split('-')
                    old_remote_switch_id = int(tmp[0])
                    old_qid = float(tmp[1])
                    #remove inbound qid from old remote switch
                    switches[old_remote_switch_id].coflow_queue[old_qid][2].remove(str(switches.index(switch))+"-"+str(c_id))
                    #update record
                    switch.coflow_queue[c_id][3] = outbound_qid
        '''     
        #Find the mapping queues on remote switches of current queue with queue id = c_id
        #queue id = switch id + "-" + coflow id
        if len(switch.coflow_queue[c_id][0])%MAPPING_THRESHOLD==0:
            print("---Beginning of Mapping---")
            for remote_switch in switches:
                if remote_switch == switch:
                    continue
                remote_cid = single_queue_mapping(switch, remote_switch, c_id)
                remote_sid = switches.index(remote_switch)
                print("Local switch id:", switches.index(switch))
                print("Remote switch id:", switches.index(remote_switch))
                print("Remote queue id:", remote_cid)
                if remote_cid == -1:
                    print("No mapping queue found, exit mapping process")
                    continue
                #uncomment this if statement for method 2: only accept mapping if the mapping result is bidirectional
                ''' 
                if single_queue_mapping(remote_switch, switch, remote_cid)!=c_id:
                    print("Remote queue doesn't map back to local queue")
                    continue
                '''
                #record remote switch queue id in local switch queue
                outbound_qid = str(remote_sid)+"-"+str(remote_cid)
                local_qid = str(switches.index(switch))+"-"+str(c_id)
                print("outbound queue id = ", outbound_qid)
                if outbound_qid in switch.coflow_queue[c_id][3]:
                    print("Current mapping result is already recorded, exit mapping process")
                    continue
                #check old qid that needs to be removed
                print("Checking if the mapping queue on current remote switch has changed")
                to_be_removed = None
                if len(switch.coflow_queue[c_id][3])>0:
                    for remote_qid in switch.coflow_queue[c_id][3]:
                        tmp = remote_qid.split('-')
                        sid = int(tmp[0])
                        cid = float(tmp[1])
                        if sid == remote_sid and cid != remote_cid:
                            to_be_removed = remote_qid
                            break
                print("to be removed: ", to_be_removed)
                if to_be_removed != None:
                    #remove old qid from local queue (outbound)
                    print("outbound queue record section of cid= ", c_id, " on local switch before removal: ", switch.coflow_queue[c_id][3])
                    switch.coflow_queue[c_id][3].remove(to_be_removed)
                    print("outbound queue record section of cid= ", c_id, " on local switch after removal: ", switch.coflow_queue[c_id][3])
                    #remove local qid from remote queue (inbound)
                    tmp = to_be_removed.split('-')
                    old_sid = int(tmp[0])
                    old_cid = float(tmp[1])
                    print("local qid that needs to be removed on remote switch=", local_qid)
                    if old_cid in remote_switch.coflow_queue.keys() and local_qid in remote_switch.coflow_queue[old_cid][2]:
                        print("inbound queue record section of cid= ", old_cid, " on remote switch sid= ", old_sid ," before removal: ", remote_switch.coflow_queue[old_cid][2])
                        switches[old_sid].coflow_queue[old_cid][2].remove(local_qid)
                        print("inbound queue record section of cid= ", old_cid, " on remote switch sid= ", old_sid ," after removal: ", remote_switch.coflow_queue[old_cid][2])
                switch.coflow_queue[c_id][3].append(outbound_qid)
                print("Store new outbound qid=", outbound_qid, "at cid=", c_id, " on local switch")
                #record local queue id in remote switch queue
                remote_switch.coflow_queue[remote_cid][2].append(local_qid)
                print("Store local qid= ",local_qid ,"at inbound record section of cid=", remote_cid," on remote switch")
            print("---End of Mapping---")

            '''
            if len(switch.coflow_queue[c_id][3])>0:
                print("---Coflow size estimation record section---")
                local_sizes = []
                major_coflow = {}
                size = 0
                for fid in switch.coflow_queue[c_id][0]:
                    if fid in switch.flow_record_table.keys():
                        size += switch.flow_record_table[fid][2]
                    cid = switch.coflow_queue[c_id][1][switch.coflow_queue[c_id][0].index(fid)]
                    if cid not in major_coflow.keys():
                        major_coflow[cid] = 1
                    else:
                        major_coflow[cid] += 1
                local_sizes.append(size)
                queue_sizes = []
                queue_sizes.append(len(switch.coflow_queue[c_id][0]))
                
                for outbound_qid in switch.coflow_queue[c_id][3]:
                    tmp = outbound_qid.split('-')
                    remote_sid = int(tmp[0])
                    remote_qid = float(tmp[1])

                    size = 0
                    for fid in switches[remote_sid].coflow_queue[remote_qid][0]:
                        if fid in switches[remote_sid].flow_record_table.keys():
                            size += switches[remote_sid].flow_record_table[fid][2]
                        cid = switches[remote_sid].coflow_queue[remote_qid][1][switches[remote_sid].coflow_queue[remote_qid][0].index(fid)]
                        if cid not in major_coflow.keys():
                            major_coflow[cid] = 1
                        else:
                            major_coflow[cid] += 1
                    local_sizes.append(size)
                    queue_sizes.append(len(switches[remote_sid].coflow_queue[remote_qid][0]))
                overall_major_coflow = max(major_coflow, key=major_coflow.get)
                global_coflow_size = current_global_coflow_size[overall_major_coflow]
                coflow_size_estimation_record[counter] = [overall_major_coflow, global_coflow_size, sum(local_sizes), local_sizes.copy(), queue_sizes.copy()]
                '''
        #---------------------


        '''
        # ------ Record ------
        coflow_distribution = groundtruth(switch, packet[0])
        representive = False
        if c_id == max(coflow_distribution, key=coflow_distribution.get):
            representive = True
        
        queue_distribution = {}
        for key in switch.coflow_queue.keys():
            queue_distribution[key] = {}
            for coflow in switch.coflow_queue[key][1]:
                if coflow not in queue_distribution[key].keys():
                    queue_distribution[key][coflow]=1
                else:
                    queue_distribution[key][coflow]+=1
        dominated = False
        if packet[0] == max(queue_distribution[c_id], key=queue_distribution[c_id].get):
            dominated = True

        qsize = 0
        total_cid_count = 0
        for cid in switch.coflow_queue[c_id][1]:
            qsize += 1
            if packet[0] == cid:
                total_cid_count += 1
        
        #["Time", "Real cid", "Classified cid", "Queue Distribution", "Representive", "Percentage", "Queue Size", "Dominated"]
        switch.classify_result_table[counter]=[packet[0], c_id, queue_distribution, representive, total_cid_count/qsize, qsize, dominated]
        # ------ Record ------
        '''
        
def mapping_score(switch, switches):
    local_index = switches.index(switch)
    remote_switch = switches[1-switches.index(switch)]
    final_result = {}
    for key in switch.coflow_queue.keys():
        final_result[key] = -1
    for key in switch.coflow_queue.keys():
        fid_list = switch.coflow_queue[key][0].copy()
        cid_list = switch.coflow_queue[key][1].copy()
        local_queue_size = len(fid_list)
        temp = list(zip(fid_list, cid_list))
        random.shuffle(temp)
        res1, res2 = zip(*temp)
        res1, res2 = list(res1), list(res2)
        sample_num = min(len(fid_list), 20)
        sample_fid_list = res1[:sample_num]
        sample_cid_list = res2[:sample_num]
        current_result = {}
        for fid in sample_fid_list:
            cnt = sketchAction(switch, fid, switch.packet_count_table, 0, False)
            if cnt==0:
                continue
            size = sketchAction(switch, fid, switch.flow_size_table, 0, False)
            packet_m = size / cnt
            arrival_t = switch.flow_record_table[fid][4]
            real_cid = sample_cid_list[sample_fid_list.index(fid)]
            result = classify(remote_switch, fid, real_cid, packet_m, arrival_t)
            all_score = result[2]
            for foreign_key in all_score.keys():
                if foreign_key not in current_result.keys():
                    current_result[foreign_key]=0
                current_result[foreign_key]+=all_score[foreign_key]
        if not current_result:
            continue
        final_result[key] = max(current_result, key=current_result.get)
    switch.mapping_result[counter] = final_result.copy()

def mapping_cid(switch, switches):
    final_result = {}
    for remote_switch in switches:
        if remote_switch == switch:
            continue
        for key in switch.coflow_queue.keys():
            fid_list = switch.coflow_queue[key][0].copy()
            cid_list = switch.coflow_queue[key][1].copy()
            local_queue_size = len(fid_list)
            temp = list(zip(fid_list, cid_list))
            random.shuffle(temp)
            res1, res2 = zip(*temp)
            res1, res2 = list(res1), list(res2)
            sample_num = min(local_queue_size, 20)
            sample_fid_list = res1[:sample_num]
            sample_cid_list = res2[:sample_num]
            current_result = {}
            for fid in sample_fid_list:
                cnt = sketchAction(switch, fid, switch.packet_count_table, 0, False)
                if cnt==0:
                    continue
                size = sketchAction(switch, fid, switch.flow_size_table, 0, False)
                packet_m = size / cnt
                arrival_t = switch.flow_record_table[fid][4]
                real_cid = sample_cid_list[sample_fid_list.index(fid)]
                cid = classify_for_mapping(remote_switch, fid, packet_m, arrival_t)
                
                if cid not in current_result.keys():
                    current_result[cid]=1
                else:
                    current_result[cid]+=1
            if not current_result:
                continue
            mapping_remote_qid = str(switches.index(remote_switch))+"-"+str(max(current_result, key=current_result.get))
            if key not in final_result.keys():
                final_result[key] = [[mapping_remote_qid], local_queue_size]
            else:
                final_result[key][0].append(mapping_remote_qid)
    
    #local queue distribution
    queue_distribution = {}
    for key in switch.coflow_queue.keys():
        queue_distribution[key] = {}
        for coflow in switch.coflow_queue[key][1]:
            if coflow not in queue_distribution[key].keys():
                queue_distribution[key][coflow]=1
            else:
                queue_distribution[key][coflow]+=1
    switch.mapping_result[counter] = [final_result, queue_distribution]

def single_queue_mapping(local_switch, remote_switch, local_qid):
    print("---Single queue mapping---")
    fid_list = local_switch.coflow_queue[local_qid][0].copy()
    print("length of original fid list: ", len(fid_list))
    local_queue_size = len(fid_list)
    random.shuffle(fid_list)
    sample_num = min(local_queue_size, 20)
    sample_fid_list = fid_list[:sample_num]
    print("length of sample fid list: ", len(sample_fid_list))
    current_result = {}
    for fid in sample_fid_list:
        print("fid = ", fid)
        if fid not in local_switch.flow_record_table.keys():
            print("fid not in local switch's flow record table")
            continue
        packet_m = local_switch.flow_record_table[fid][5]
        arrival_t = local_switch.flow_record_table[fid][4]
        print("Packet_m = ", packet_m)
        print("Arrival_t = ", arrival_t)
        cid = classify_for_mapping(remote_switch, packet_m, arrival_t) 
        print("cid = ", cid)
        '''
        if cid == -1:
            continue
        '''
        if cid not in current_result.keys():
            current_result[cid]=1
        else:
            current_result[cid]+=1
    print("current result = ", current_result)
    print("---End of single queue mapping---")
    if len(current_result)==0:
        return -1
    else:
        return max(current_result, key=current_result.get)
    
def mapping_to_grouping(switches):
    def dfs(group, major, switches, switch_index, qid):
        group.append(str(switch_index)+"-"+str(qid))
        for next_hop in switches[switch_index].mapping_result[counter][0][qid][0]:
            tmp = next_hop.split("-")
            next_hop_switch_index = int(tmp[0])
            next_hop_qid = float(tmp[1])
            
            if next_hop_qid in switches[next_hop_switch_index].coflow_queue.keys() and str(next_hop_switch_index)+"-"+str(next_hop_qid) not in group:
                if str(switch_index)+"-"+str(qid) in switches[next_hop_switch_index].mapping_result[counter][0][next_hop_qid][0]:
                    major.append(str(switch_index)+"-"+str(qid))
                    major.append(str(next_hop_switch_index)+"-"+str(next_hop_qid))
                dfs(group, major, switches, next_hop_switch_index, next_hop_qid)
        return
    
    mapping_to_grouping_table[counter] = []
    visited = []
    for switch in switches:
        for qid in switch.coflow_queue.keys():
            group = []
            major = []
            if str(switches.index(switch))+"-"+str(qid) not in visited:
                dfs(group, major, switches, switches.index(switch), qid)
            mapping_to_grouping_table[counter].append([group, major])
            for q in group:
                visited.append(q)
    return

def schedule(table):
    for c_id in table.keys():
        size = table[c_id][0] * 1024 ##
        curPriority = 0
        tmp = INIT_QUEUE_LIMIT
        while size > tmp:
            curPriority += 1
            tmp *= JOB_SIZE_MULT
            if curPriority >= NUM_JOB_QUEUES:
                break
        table[c_id][1] = curPriority
    return table

def controllerUpdate(switch, switches):
    # Update Flow Table (Size)
    coflow_size = {} # Coflow id, coflow size 
    for f_id in switch.flow_record_table.keys():
        if switch.flow_record_table[f_id][6] == False: #if flow not finished
            size = sketchAction(switch, f_id, switch.flow_size_table, 0, False)
            switch.flow_record_table[f_id][2] = size #update flow size
        # For next step
        if switch.flow_record_table[f_id][0] != None: #if flow has coflow id
            if switch.flow_record_table[f_id][0] not in coflow_size.keys():
                coflow_size[switch.flow_record_table[f_id][0]] = switch.flow_record_table[f_id][2]
            else:
                coflow_size[switch.flow_record_table[f_id][0]] += switch.flow_record_table[f_id][2]  
    # Update coflow size
    for c_id in switch.coflow_priority_table.keys():
        if c_id not in coflow_size.keys(): # Bug
            continue
        switch.coflow_priority_table[c_id][0] = coflow_size[c_id]
        # Add remote queue size
        if len(switch.coflow_queue[c_id][2])>0:
            to_be_removed = []
            for inbound_size in switch.coflow_queue[c_id][2]:
                tmp = inbound_size.split('-')
                remote_switch_id = int(tmp[0])
                remote_queue_id = float(tmp[1])
                if remote_queue_id in switches[remote_switch_id].coflow_queue.keys():
                    for remote_fid in switches[remote_switch_id].coflow_queue[remote_queue_id][0]:
                        if remote_fid in switches[remote_switch_id].flow_record_table.keys():
                            switch.coflow_priority_table[c_id][0] += switches[remote_switch_id].flow_record_table[remote_fid][2]
                else:
                    to_be_removed.append(inbound_size)
            for item in to_be_removed:
                switch.coflow_queue[c_id][2].remove(item)

    # Schedule
    switch.coflow_priority_table = schedule(switch.coflow_priority_table) # Update coflow priority
    # print("Coflow Table", coflow_priority_table)
    # Update Flow Table (Priority)
    update_flow_list = [] # Flow ID, Priority
    for f_id in switch.flow_record_table.keys():
        if switch.flow_record_table[f_id][0] != None: # Classified
            if switch.flow_record_table[f_id][1] != switch.coflow_priority_table[switch.flow_record_table[f_id][0]][1]: # Update priority
                switch.flow_record_table[f_id][1] = switch.coflow_priority_table[switch.flow_record_table[f_id][0]][1]
                update_flow_list.append([f_id, switch.flow_record_table[f_id][1]])
    # Update Priority Table
    for entry in update_flow_list:
        if entry[0] not in switch.priority_table.keys():
            #print("(Update priority in Priority Table) Flow ", f_id, " is not in Priority Table")
            if len(switch.priority_table) < PRIORITY_TABLE_SIZE:
                switch.priority_table[f_id] = entry[1]
            else:
                print("(Priority Table) Overflow")
                # Todo
        else:
            switch.priority_table[entry[0]] = entry[1]
    return switch.coflow_priority_table

def controllerUpdateTTL(switch, f_id):
    clear_now = []
    finished_coflow = {}
    # Update TTL
    for f in switch.flow_record_table.keys(): 
        if f == f_id:
            switch.flow_record_table[f_id][3] = INITIAL_TTL
            switch.flow_record_table[f][6] = False
        else:
            switch.flow_record_table[f][3] -= 1
            if switch.flow_record_table[f][3] <= 0 and switch.flow_record_table[f][6] == False:
                #print(counter, " ############### Clear", f_id)
                if switch.flow_record_table[f][0] == None: 
                    sketchAction(switch, f, switch.packet_count_table, 0, True)
                    clear_now.append(f)
                else: # Classified
                    if f in switch.priority_table.keys():
                        del switch.priority_table[f]
                        sketchAction(switch, f, switch.flow_size_table, 0, True)
                    switch.flow_record_table[f][6] = True
        if switch.flow_record_table[f][0] != None:
            if switch.flow_record_table[f][0] not in finished_coflow.keys():
                finished_coflow[switch.flow_record_table[f][0]] = True
            if switch.flow_record_table[f][6] == False: # Flow unfinished
                finished_coflow[switch.flow_record_table[f][0]] = False # Coflow unfinished
    # Delete finished coflows
    for c_id in finished_coflow.keys(): 
        if finished_coflow[c_id] == True:
            del switch.coflow_priority_table[c_id]
            for f in set(switch.coflow_queue[c_id][0]):
                if f in switch.flow_record_table.keys():
                    del switch.flow_record_table[f]
            del switch.coflow_queue[c_id]
    # Delete finished flows       
    for f in clear_now:
        del switch.flow_record_table[f]
    return
    
def PIFO(packet, wait_queue):
    #print("PIFO -> packet = ", packet)
    #print("PIFO -> packet type : ", type(packet))
    wait_queue.put((packet[-1],packet))
    #print("PIFO -> after put : ", wait_queue.get())
    return wait_queue

def egress(switch):
    item = switch.wait_queue.get()
    out_packet = item[1]
    switch.output_queue.append(out_packet)
    # ------ Record ------
    if out_packet[0] not in switch.coflow_completion.keys():
        switch.coflow_completion[out_packet[0]] = [counter, counter, 0, fb_coflow_size[str(out_packet[0])], fb_coflow_priority[str(out_packet[0])]]
    else:
        switch.coflow_completion[out_packet[0]][1] = counter
        switch.coflow_completion[out_packet[0]][2] = counter - switch.coflow_completion[out_packet[0]][0]
    #current_global_coflow_size[out_packet[0]] -= out_packet[6]
    # ------ Record ------
    return switch.output_queue

def mergeCCT(switches):
    mergedCCT = {}
    for switch in switches:
        for key in switch.coflow_completion.keys():
            if key not in mergedCCT.keys():
                mergedCCT[key] = [switch.coflow_completion[key][0], switch.coflow_completion[key][1]]
            else:
                mergedCCT[key][0] = min(mergedCCT[key][0], switch.coflow_completion[key][0])
                mergedCCT[key][1] = max(mergedCCT[key][1], switch.coflow_completion[key][1])
    CCT=0
    for key in mergedCCT.keys():
        mergedCCT[key].append(mergedCCT[key][1]-mergedCCT[key][0])
        CCT = CCT+(mergedCCT[key][1]-mergedCCT[key][0])
    averageCCT = CCT/len(mergedCCT)
    mergedCCT["AverageCCT"]=averageCCT
    return mergedCCT

def priority_assign(switches):
    result = []
    
    #global priority
    global_priority = PriorityQueue()
    global_result = []
    for k, v in current_global_coflow_size.items():
        global_priority.put((v, k))
    for i in range(global_priority.qsize()):
        tmp = global_priority.get()
        global_result.append((tmp[1],tmp[0]))#(cid, size)
    
    #---print global coflow size in the order of priority---
    print("Global coflow size in the order of priority:", global_result)
    #---end of printing---

    result.append(global_result)
    result.append([])#for mapping priority
    result.append([])#for local priority

    #local priority with and without mapping
    for switch in switches:
        local_priority = PriorityQueue()
        local_result = []
        mapping_priority = PriorityQueue()
        mapping_result = []
        coflow_distribution_local = {}
        coflow_estimation_size_local = {}

        #calculate coflow distribution on local switch
        for qid in switch.coflow_queue.keys():
            for real_cid in switch.coflow_queue[qid][1]:
                if real_cid not in coflow_distribution_local.keys():
                    coflow_distribution_local[real_cid] = {}
                if qid not in coflow_distribution_local[real_cid].keys():
                    coflow_distribution_local[real_cid][qid] = 1
                else:
                    coflow_distribution_local[real_cid][qid] += 1

        #---Check coflow_distribution_local---
        print("Coflow distribution on local switch", switches.index(switch))
        for key in coflow_distribution_local.keys():
            print(key,":",coflow_distribution_local[key])
        #---end of printing---

        #calculate priority based on coflow distribution
        #Step 1: calculate coflow's major queue size
        for cid in coflow_distribution_local.keys():
            major_queue = max(coflow_distribution_local[cid], key=coflow_distribution_local[cid].get)
            print(cid,"'s major queue:", major_queue)

            #calculate the coflow size based on its major queue
            size = 0
            for fid in switch.coflow_queue[major_queue][0]:
                if fid in switch.flow_record_table.keys():
                    size += switch.flow_record_table[fid][2]
            coflow_estimation_size_local[cid] = size
        
        #---Check coflow estimation size---
        print("Coflow estimation size on local switch", switches.index(switch))
        for key in coflow_estimation_size_local.keys():
            print(key, ":", coflow_estimation_size_local[key])
        #---end of printing---

        #Step 2: calculate priority according to coflow's major queue size
        for cid, size in coflow_estimation_size_local.items():
            local_priority.put((size, cid))
        for i in range(local_priority.qsize()):
            tmp = local_priority.get()
            local_result.append((tmp[1],tmp[0])) #(cid, size)
        #Step 3: Store the result
        result[2].append(local_result)

        #---print local coflow size without mapping in the order of priority---
        print("Local coflow size without mapping in the order of priority on switch", switches.index(switch))
        print(local_result)
        #---end of printing---

        #calculate coflow distribution on local switch with mapping
        coflow_distribution_mapping = coflow_distribution_local.copy()
        for qid in switch.coflow_queue.keys():
            if len(switch.coflow_queue[qid][3])>0:
                for remote_id in switch.coflow_queue[qid][3]:
                    tmp = remote_id.split('-')
                    remote_sid = int(tmp[0])
                    remote_qid = float(tmp[1])
                    if remote_qid in switches[remote_sid].coflow_queue.keys():
                        for remote_cid in switches[remote_sid].coflow_queue[remote_qid][1]:
                            if remote_cid not in coflow_distribution_mapping.keys():
                                continue
                            if qid not in coflow_distribution_mapping[remote_cid].keys():
                                coflow_distribution_mapping[remote_cid][qid] = 1
                            else:
                                coflow_distribution_mapping[remote_cid][qid] += 1
            else:
                print("Queue", qid, "on switch", switches.index(switch), "doesn't have outbound mapping queue")

        #---Check coflow_distribution_mapping---
        print("Coflow distribution on local switch", switches.index(switch), "with mapping")
        for key in coflow_distribution_mapping.keys():
            print(key,":",coflow_distribution_mapping[key])
        #---end of printing---

        #calculate priority based on coflow distribution
        coflow_estimation_size_mapping = {}
        #Step 1: calculate coflow's major queue size plus the remote queue size
        for cid in coflow_distribution_mapping.keys():
            major_queue = max(coflow_distribution_mapping[cid], key=coflow_distribution_mapping[cid].get)
            print(cid,"'s major queue:", major_queue)

            #calculate the coflow local size
            local_size = 0
            for fid in switch.coflow_queue[major_queue][0]:
                if fid in switch.flow_record_table.keys():
                    local_size += switch.flow_record_table[fid][2]
            coflow_estimation_size_mapping[cid] = local_size

            #calculate the coflow remote size
            remote_size = 0
            if len(switch.coflow_queue[major_queue][3])>0:
                for remote_id in switch.coflow_queue[major_queue][3]:
                    tmp = remote_id.split('-')
                    remote_sid = int(tmp[0])
                    remote_qid = float(tmp[1])
                    if remote_qid in switches[remote_sid].coflow_queue.keys():
                        for remote_fid in switches[remote_sid].coflow_queue[remote_qid][0]:
                            if remote_fid in switches[remote_sid].flow_record_table.keys():
                                remote_size += switches[remote_sid].flow_record_table[remote_fid][2]
            coflow_estimation_size_mapping[cid] += remote_size
        
        #---Check coflow estimation size with mapping---
        print("Coflow estimation size with mapping on local switch", switches.index(switch))
        for key in coflow_estimation_size_mapping.keys():
            print(key, ":", coflow_estimation_size_mapping[key])
        #---end of printing---

        #Step 2: calculate priority according to coflow's major queue size
        for cid, size in coflow_estimation_size_mapping.items():
            mapping_priority.put((size, cid))
        for i in range(mapping_priority.qsize()):
            tmp = mapping_priority.get()
            mapping_result.append((tmp[1],tmp[0]))#cid, size
        #Step 3: Store the result
        result[1].append(mapping_result)

        #---print mapping result in the order of priority---
        print("Mapping result on switch", switches.index(switch))
        print(mapping_result)

    return result

In [3]:
readDataSet()

In [ ]:
print("Read packets data: ")
input_queue, input_data_flow, f_id_list, c_list = loadCsvData()
print(len(c_list), " coflows, ", len(f_id_list), " flows and ", len(input_queue), " packets")

Read packets data: 
open  1_30_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  31_50_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  51_70_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  71_90_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  91_120_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  121_150_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  151_170_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv h012.csv h013.csv h014.csv h015.csv h016.csv 
open  171_200_M1000_50_V10_1_420_delay
h009.csv h010.csv h011.csv 

In [169]:
#sampling
sample_limit = 3000000
numofCoflows = 30
sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list = sampling(input_queue, input_data_flow, f_id_list, c_list, numofCoflows)


while len(sample_input_queue)>sample_limit:
   sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list = sampling(input_queue, input_data_flow, f_id_list, c_list, numofCoflows)

print("After sampling: ")
print(len(sample_c_list), " coflows, ", len(sample_f_id_list), " flows and ", len(sample_input_queue), " packets")

After sampling: 
30  coflows,  33680  flows and  232508  packets


In [159]:
for key in global_coflow_size.keys():
    print("key=",key)
    print(type(key))
    print("value=",global_coflow_size[key])
    print(type(global_coflow_size[key]))

key= 34.0
<class 'numpy.float64'>
value= 3962492.0
<class 'numpy.float64'>
key= 33.0
<class 'numpy.float64'>
value= 3020954.0
<class 'numpy.float64'>
key= 31.0
<class 'numpy.float64'>
value= 2996910.0
<class 'numpy.float64'>
key= 32.0
<class 'numpy.float64'>
value= 5213098.0
<class 'numpy.float64'>
key= 35.0
<class 'numpy.float64'>
value= 2587168.0
<class 'numpy.float64'>
key= 36.0
<class 'numpy.float64'>
value= 2593114.0
<class 'numpy.float64'>
key= 37.0
<class 'numpy.float64'>
value= 1076386.0
<class 'numpy.float64'>
key= 38.0
<class 'numpy.float64'>
value= 1062102.0
<class 'numpy.float64'>
key= 39.0
<class 'numpy.float64'>
value= 1218344.0
<class 'numpy.float64'>
key= 40.0
<class 'numpy.float64'>
value= 2717754.0
<class 'numpy.float64'>
key= 44.0
<class 'numpy.float64'>
value= 958726.0
<class 'numpy.float64'>
key= 42.0
<class 'numpy.float64'>
value= 4208552.0
<class 'numpy.float64'>
key= 45.0
<class 'numpy.float64'>
value= 1157984.0
<class 'numpy.float64'>
key= 41.0
<class 'numpy.fl

In [181]:
#grouping
switches=[]
numOfSwitches = 2
for i in range(numOfSwitches):
    switches.append(Switch())
#switches = fair_grouping(switches, sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list, numOfSwitches)
switches = grouping(switches, sample_input_queue, sample_input_data_flow, sample_f_id_list, numOfSwitches)
#switches, shuffle_cid_list = grouping2(switches, sample_input_queue, sample_input_data_flow, sample_f_id_list, sample_c_list, numOfSwitches)
for switch in switches:
    print(len(switch.input_queue))

109869
122639


In [182]:
global_reset()

In [ ]:
counter=0
#mapping_counter=35000
packet_index=-1
while True:
    counter+=1
    packet_index+=1
    done=0
    for switch in switches:
        if counter >= len(switch.input_queue) and switch.wait_queue.qsize() == 0:
            done+=1
            continue
        if packet_index < len(switch.input_queue):
            this_packet = list(switch.input_queue[packet_index])
            if this_packet[0] not in current_global_coflow_size.keys():
                current_global_coflow_size[this_packet[0]] = this_packet[6]
            else:
                current_global_coflow_size[this_packet[0]] += this_packet[6]
            f_id = getFlowID(this_packet, sample_f_id_list)
            find, this_packet = checkPriorityTable(switch, f_id, this_packet)
            if not find:
                # Update Packet Count Table
                action = updatePacketCntTable(switch, f_id, this_packet)
            # Update Flow Size Table
            updateFlowSizeTable(switch, f_id, this_packet)
            # New flow or Packet full, inform controller
            if not find and action:
                updateFlowRecordTable(switches, switch, f_id, sample_c_list, this_packet)
            #print("flow record table: ", switch.flow_record_table)
            # Controller update
            if counter % CONTROLLER_UPDATE_TIME == 0 or packet_index == len(switch.input_queue)-1:
                switch.coflow_priority_table = controllerUpdate(switch, switches)
            #PIFO
            switch.wait_queue = PIFO(this_packet, switch.wait_queue)
            '''
            if current_global_coflow_size[real_cid]>=global_coflow_size[real_cid]/2 and real_cid not in priority_done:
                result = priority_assign(switches)
                print("Counter = ", counter)
                print("Priority assignment result = ", result)
                priority_result[counter] = [real_cid, result]
                priority_done.add(real_cid)
            '''
            if counter % PRIORITY_ASSIGNMENT_RATE == 0:
                result = priority_assign(switches)
                print("Switch sid=", switches.index(switch))
                print("Counter = ", counter)
                print("Priority assignment result = ", result)
                key = str(switches.index(switch))+"-"+str(counter)
                priority_result[key] = result
        # Egress
        if counter % (EGRESS_RATE*numOfSwitches) == 0:
            switch.output_queue = egress(switch)
        '''
        # Print Result
        if counter % 100 == 0:
            print("Switch ", switches.index(switch))
            print("Time slot: ", counter)
            print("Size of Priority Table: ", len(switch.priority_table.keys()))
            switch.priority_table_time.append(counter)
            switch.priority_table_size.append(len(switch.priority_table.keys()))
            if switch.DNN_counter != 0:
                print("DNN Accuracy: ", switch.DNN_right / switch.DNN_counter * 100, " %")
            if switch.sketch_counter != 0:
                print("Sketch Count Err: ", switch.sketch_cnt_err / switch.sketch_counter * 100, " %")
                print("Sketch Size Err: ", switch.sketch_size_err / switch.sketch_counter * 100, " %")
                print("Sketch Mean Err: ", switch.sketch_mean_err / switch.sketch_counter * 100, " %")
            print("len of wait queue: ", len(switch.wait_queue.queue))
        '''
        # Update TTL
        controllerUpdateTTL(switch, f_id)
    '''
    if counter>=mapping_counter:
        for switch in switches:
            mapping_cid(switch,switches)
        #mapping_to_grouping(switches)
        mapping_counter+=35000
    '''
    if done==numOfSwitches:
        break
print("All switches complete")

(cnt =  20 ) Flow  9260  is not in Flow Table
(cnt =  20 ) Flow  16109  is not in Flow Table
(cnt =  20 ) Flow  17148  is not in Flow Table
(cnt =  20 ) Flow  14375  is not in Flow Table
(cnt =  20 ) Flow  20273  is not in Flow Table
(cnt =  20 ) Flow  19810  is not in Flow Table
(cnt =  20 ) Flow  5544  is not in Flow Table
(cnt =  20 ) Flow  12790  is not in Flow Table
(cnt =  20 ) Flow  30529  is not in Flow Table
(cnt =  20 ) Flow  20021  is not in Flow Table
(cnt =  20 ) Flow  2340  is not in Flow Table
(cnt =  20 ) Flow  5268  is not in Flow Table
(cnt =  20 ) Flow  27200  is not in Flow Table
(cnt =  20 ) Flow  20376  is not in Flow Table
(cnt =  20 ) Flow  898  is not in Flow Table
(cnt =  20 ) Flow  31793  is not in Flow Table
(cnt =  20 ) Flow  882  is not in Flow Table
(cnt =  20 ) Flow  20303  is not in Flow Table
(cnt =  20 ) Flow  6586  is not in Flow Table
(cnt =  20 ) Flow  25652  is not in Flow Table
(cnt =  20 ) Flow  21502  is not in Flow Table
(cnt =  20 ) Flow  186

cid =  -1
fid =  12177
Packet_m =  208.8
Arrival_t =  1597194066.521811
cid =  -1
fid =  13311
Packet_m =  39.2
Arrival_t =  1597194384.307682
cid =  -1
fid =  5501
Packet_m =  125.6
Arrival_t =  1597194137.551778
cid =  -1
fid =  6431
Packet_m =  81.25
Arrival_t =  1597194262.195775
cid =  -1
fid =  10908
Packet_m =  104.15
Arrival_t =  1597194445.258719
cid =  -1
fid =  12009
Packet_m =  58.8
Arrival_t =  1597194117.729892
cid =  -1
fid =  3958
Packet_m =  84.15
Arrival_t =  1597194347.683318
cid =  -1
fid =  13228
Packet_m =  167.0
Arrival_t =  1597194707.117663
cid =  -1
fid =  3088
Packet_m =  349.35
Arrival_t =  1597192678.597504
cid =  -1
fid =  5342
Packet_m =  104.65
Arrival_t =  1597193385.224057
cid =  -1
fid =  14470
Packet_m =  177.1
Arrival_t =  1597192887.444044
cid =  -1
fid =  11508
Packet_m =  20.05
Arrival_t =  1597192649.373168
cid =  -1
fid =  11812
Packet_m =  253.1
Arrival_t =  1597194706.398899
cid =  -1
fid =  15746
Packet_m =  811.0
Arrival_t =  1597192960.485

(cnt =  20 ) Flow  5247  is not in Flow Table
(cnt =  20 ) Flow  8165  is not in Flow Table
(cnt =  20 ) Flow  3407  is not in Flow Table
(cnt =  20 ) Flow  13017  is not in Flow Table
(cnt =  20 ) Flow  141  is not in Flow Table
(cnt =  20 ) Flow  14239  is not in Flow Table
(cnt =  20 ) Flow  863  is not in Flow Table
(cnt =  20 ) Flow  11613  is not in Flow Table
(cnt =  20 ) Flow  4839  is not in Flow Table
(cnt =  20 ) Flow  9250  is not in Flow Table
(cnt =  20 ) Flow  4681  is not in Flow Table
(cnt =  20 ) Flow  2641  is not in Flow Table
(cnt =  20 ) Flow  2647  is not in Flow Table
(cnt =  20 ) Flow  3019  is not in Flow Table
(cnt =  20 ) Flow  3598  is not in Flow Table
(cnt =  20 ) Flow  29  is not in Flow Table
(cnt =  20 ) Flow  2012  is not in Flow Table
(cnt =  20 ) Flow  6102  is not in Flow Table
(cnt =  20 ) Flow  2912  is not in Flow Table
(cnt =  20 ) Flow  12648  is not in Flow Table
(cnt =  20 ) Flow  5012  is not in Flow Table
(cnt =  20 ) Flow  5529  is not in

(cnt =  20 ) Flow  32362  is not in Flow Table
(cnt =  20 ) Flow  32895  is not in Flow Table
(cnt =  20 ) Flow  33472  is not in Flow Table
(cnt =  20 ) Flow  20870  is not in Flow Table
(cnt =  20 ) Flow  21370  is not in Flow Table
(cnt =  20 ) Flow  19204  is not in Flow Table
(cnt =  20 ) Flow  15580  is not in Flow Table
(cnt =  20 ) Flow  13654  is not in Flow Table
(cnt =  20 ) Flow  9439  is not in Flow Table
(cnt =  20 ) Flow  12557  is not in Flow Table
(cnt =  20 ) Flow  4702  is not in Flow Table
(cnt =  20 ) Flow  15016  is not in Flow Table
(cnt =  20 ) Flow  27408  is not in Flow Table
(cnt =  20 ) Flow  7854  is not in Flow Table
(cnt =  20 ) Flow  12687  is not in Flow Table
(cnt =  20 ) Flow  11247  is not in Flow Table
(cnt =  20 ) Flow  8657  is not in Flow Table
(cnt =  20 ) Flow  15129  is not in Flow Table
(cnt =  20 ) Flow  1418  is not in Flow Table
(cnt =  20 ) Flow  8107  is not in Flow Table
(cnt =  20 ) Flow  10992  is not in Flow Table
(cnt =  20 ) Flow  

cid =  -1
fid =  27589
Packet_m =  43.2
Arrival_t =  1597634856.022798
cid =  -1
fid =  22480
Packet_m =  174.8
Arrival_t =  1597635588.946805
cid =  -1
fid =  22107
Packet_m =  430.85
Arrival_t =  1597636281.010856
cid =  -1
fid =  24082
Packet_m =  262.5
Arrival_t =  1597636143.653838
cid =  206.0
fid =  25718
Packet_m =  87.75
Arrival_t =  1597634150.677835
cid =  -1
current result =  {-1: 16, 206.0: 4}
---End of single queue mapping---
Local switch id: 1
Remote switch id: 0
Remote queue id: -1
outbound queues =  []
(cnt =  20 ) Flow  9120  is not in Flow Table
(cnt =  20 ) Flow  27973  is not in Flow Table
(cnt =  20 ) Flow  16318  is not in Flow Table
(cnt =  20 ) Flow  9860  is not in Flow Table
(cnt =  20 ) Flow  5456  is not in Flow Table
(cnt =  20 ) Flow  5785  is not in Flow Table
(cnt =  20 ) Flow  23398  is not in Flow Table
(cnt =  20 ) Flow  3861  is not in Flow Table
(cnt =  20 ) Flow  8621  is not in Flow Table
---Beginning of Method 2---
---Single queue mapping---
len

...

cid =  286.0
fid =  19914
Packet_m =  1238.3
Arrival_t =  1604492328.906269
cid =  286.0
fid =  33629
Packet_m =  956.8
Arrival_t =  1604492274.867827
cid =  286.0
fid =  30955
Packet_m =  270.0
Arrival_t =  1604491586.862272
cid =  286.0
fid =  26306
Packet_m =  830.75
Arrival_t =  1604492259.274552
cid =  286.0
fid =  25550
Packet_m =  716.3
Arrival_t =  1604491482.150039
cid =  286.0
fid =  30571
Packet_m =  830.1
Arrival_t =  1604491430.970321
cid =  286.0
fid =  20029
Packet_m =  1000.2
Arrival_t =  1602383154.042708
cid =  286.0
current result =  {286.0: 20}
---End of single queue mapping---
Local switch id: 1
Remote switch id: 0
Remote queue id: 286.0
---Single queue mapping---
length of original fid list:  149
length of sample fid list:  20
fid =  5789
Packet_m =  1303.75
Arrival_t =  1604491648.910237
cid =  333.0
fid =  9048
Packet_m =  663.45
Arrival_t =  1604491440.791913
cid =  333.0
fid =  5695
Packet_m =  535.0
Arrival_t =  1604492915.255813
cid =  333.0
fid =  2327
Pack

cid =  288.0
fid =  29742
Packet_m =  801.8
Arrival_t =  1604491463.30785
cid =  288.0
fid =  20805
Packet_m =  270.0
Arrival_t =  1604492346.755209
cid =  288.0
fid =  20507
Packet_m =  135.75
Arrival_t =  1604491387.498483
cid =  288.0
fid =  31263
Packet_m =  787.8
Arrival_t =  1604491416.252047
cid =  288.0
fid =  28104
Packet_m =  894.4
Arrival_t =  1604491410.220558
cid =  288.0
fid =  23571
Packet_m =  828.8
Arrival_t =  1602381053.881
cid =  288.0
fid =  32567
Packet_m =  1043.35
Arrival_t =  1602739051.254382
cid =  288.0
fid =  25149
Packet_m =  1290.85
Arrival_t =  1602739002.342333
cid =  288.0
fid =  32879
Packet_m =  530.0
Arrival_t =  1604491695.092195
cid =  288.0
fid =  28555
Packet_m =  268.5
Arrival_t =  1604492316.808777
cid =  288.0
fid =  22288
Packet_m =  1057.95
Arrival_t =  1604491402.31289
cid =  288.0
fid =  22371
Packet_m =  624.5
Arrival_t =  1604492260.266105
cid =  288.0
fid =  19775
Packet_m =  852.55
Arrival_t =  1604491497.912718
cid =  288.0
current r

---Beginning of Method 2---
---Single queue mapping---
length of original fid list:  150
length of sample fid list:  20
fid =  7435
Packet_m =  518.0
Arrival_t =  1604493277.642079
cid =  353.0
fid =  3087
Packet_m =  1083.5
Arrival_t =  1604491978.265945
cid =  353.0
fid =  8700
Packet_m =  797.65
Arrival_t =  1600517833.821107
cid =  353.0
fid =  13999
Packet_m =  530.0
Arrival_t =  1604493305.425375
cid =  353.0
fid =  9104
Packet_m =  1064.6
Arrival_t =  1604492138.56195
cid =  353.0
fid =  11213
Packet_m =  949.3
Arrival_t =  1604491460.849828
cid =  353.0
fid =  14641
Packet_m =  869.25
Arrival_t =  1602383418.083013
cid =  353.0
fid =  12746
Packet_m =  932.45
Arrival_t =  1604491384.261737
cid =  353.0
fid =  2891
Packet_m =  720.6
Arrival_t =  1604492668.739153
cid =  353.0
fid =  6670
Packet_m =  269.0
Arrival_t =  1604492154.358069
cid =  353.0
fid =  10104
Packet_m =  789.4
Arrival_t =  1604493105.825222
cid =  353.0
fid =  4654
Packet_m =  354.7
Arrival_t =  1604492704.265

cid =  360.0
fid =  156
Packet_m =  612.3
Arrival_t =  1604491395.848844
cid =  360.0
fid =  12986
Packet_m =  435.4
Arrival_t =  1604493107.812292
cid =  360.0
fid =  4441
Packet_m =  266.0
Arrival_t =  1604492284.86702
cid =  360.0
fid =  5467
Packet_m =  1082.55
Arrival_t =  1604492469.264812
cid =  360.0
fid =  5926
Packet_m =  534.0
Arrival_t =  1604491434.564712
cid =  360.0
current result =  {360.0: 20}
---End of single queue mapping---
Local switch id: 0
Remote switch id: 1
Remote queue id: 360.0
---Single queue mapping---
length of original fid list:  99
length of sample fid list:  20
fid =  23291
Packet_m =  1401.75
Arrival_t =  1604491459.262258
cid =  348.0
fid =  25425
Packet_m =  1552.9
Arrival_t =  1604492310.824702
cid =  348.0
fid =  17273
Packet_m =  979.65
Arrival_t =  1604491676.098266
cid =  348.0
fid =  22214
Packet_m =  1095.5
Arrival_t =  1602451575.789586
cid =  348.0
fid =  17294
Packet_m =  615.2
Arrival_t =  1604492120.526015
cid =  348.0
fid =  25233
Packet

cid =  362.0
fid =  29726
Packet_m =  267.5
Arrival_t =  1604491522.467181
cid =  362.0
fid =  18153
Packet_m =  1147.2
Arrival_t =  1604491579.180292
cid =  362.0
fid =  31662
Packet_m =  883.2
Arrival_t =  1604491380.268094
cid =  362.0
fid =  30357
Packet_m =  827.85
Arrival_t =  1604492265.543733
cid =  362.0
fid =  31749
Packet_m =  964.9
Arrival_t =  1604491410.986382
cid =  362.0
fid =  21405
Packet_m =  611.1
Arrival_t =  1604492194.999805
cid =  362.0
fid =  29588
Packet_m =  769.7
Arrival_t =  1604491433.087594
cid =  362.0
fid =  26398
Packet_m =  1574.25
Arrival_t =  1602465204.899523
cid =  362.0
fid =  27202
Packet_m =  761.85
Arrival_t =  1604491442.637158
cid =  362.0
fid =  19580
Packet_m =  704.7
Arrival_t =  1604491868.032588
cid =  362.0
fid =  26879
Packet_m =  2599.2
Arrival_t =  1604491421.129651
cid =  362.0
current result =  {362.0: 20}
---End of single queue mapping---
Remote queue doesn't map back to local queue
outbound queues =  []
(cnt =  20 ) Flow  33643 

In [179]:
for key in priority_result.keys():
    print(key, ":", priority_result[key])

742 : [1.0, [[(1.0, 197969.0), (9.0, 686829.0)], [[], [(1.0, 196545.0)]], [[], [(1.0, 196545.0)]]]]
2325 : [9.0, [[(18.0, 558.0), (1.0, 198446.0), (9.0, 2721671.0)], [[(9.0, 41036.0)], [(9.0, 12873.0), (1.0, 199735.0)]], [[(9.0, 41036.0)], [(9.0, 12873.0), (1.0, 199735.0)]]]]
4908 : [18.0, [[(18.0, 154730.0), (88.0, 305857.0), (1.0, 329844.0), (9.0, 5203025.0)], [[(88.0, 43878.0), (9.0, 107429.0)], [(9.0, 81662.0), (18.0, 150824.0), (1.0, 328451.0)]], [[(88.0, 43878.0), (9.0, 107429.0)], [(9.0, 81662.0), (18.0, 150824.0), (1.0, 328451.0)]]]]
5883 : [88.0, [[(18.0, 308783.0), (1.0, 395835.0), (88.0, 773803.0), (9.0, 5442938.0)], [[(9.0, 111047.0), (88.0, 134632.0)], [(88.0, 809.0), (9.0, 101083.0), (18.0, 318448.0), (1.0, 398401.0)]], [[(9.0, 111047.0), (88.0, 134632.0)], [(88.0, 809.0), (9.0, 101083.0), (18.0, 318448.0), (1.0, 398401.0)]]]]
7614 : [165.0, [[(128.0, 44399.0), (18.0, 308783.0), (1.0, 395835.0), (165.0, 1000884.0), (88.0, 1547048.0), (9.0, 5442938.0)], [[(9.0, 111047.0), 

In [180]:
with open("P4_RECORD/Coflow_Priority_Result"+"_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow_"+str(len(sample_input_queue))+"packets.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["timer", "cid", "global", "local with mapping", "local"])
        for k, v in priority_result.items():
            tmp = [k]
            tmp.append(priority_result[k][0])
            for item in priority_result[k][1]:
                tmp.append(item)
            writer.writerow(tmp)

In [8]:
for switch in switches:
    with open("mapping_result"+str(switches.index(switch))+".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        for k, v in switch.mapping_result.items():
            tmp = [k]
            tmp.extend(v)
            writer.writerow(tmp)

In [27]:
# ------ Record ------
for switch in switches:
    with open(OUTPUT_CSV+str(switches.index(switch)), "w", newline="") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(["Time slot", "Size"])
        for i in range(len(switch.priority_table_time)):
            writer.writerow([switch.priority_table_time[i], switch.priority_table_size[i]])
        print("Write Completed")
    with open(OUTPUT_COMPLETION_TIME+str(switches.index(switch)), "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Coflow ID", "Start Time", "Completion Time", "Duration Time", "Coflow Size", "Coflow Priority"])
        for k, v in switch.coflow_completion.items():
            tmp = [k]
            tmp.extend(v)
            writer.writerow(tmp)
# ------ Record ------

Write Completed
Write Completed


In [11]:
for switch in switches:
    with open(OUTPUT_COMPLETION_TIME+"_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow"+"_"+str(switches.index(switch))+"_"+str(len(sample_input_queue))+"packets.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Coflow ID", "Start Time", "Completion Time", "Duration Time", "Coflow Size", "Coflow Priority"])
        for k, v in switch.coflow_completion.items():
            tmp = [k]
            tmp.extend(v)
            writer.writerow(tmp)

In [15]:
for switch in switches:
    with open("P4_RECORD/classify_accuracy"+"_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow"+"_"+str(switches.index(switch))+".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(cid_parition[switches.index(switch)].keys())
        writer.writerow(["Time", "Coflow id", "Major Accuracy", "Major Groundtruth", "Minor Accuracy", "Minor Groundtruth"])
        for k, v in switch.classify_accuracy_table.items():
            tmp = [k]
            tmp.extend(v)
            writer.writerow(tmp)

In [16]:
mergedCCT = mergeCCT(switches)
print(mergedCCT)
with open("P4_RECORD/AverageCCT"+"_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow_"+str(len(sample_input_queue))+"packets.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Coflow ID", "Start Time", "Completion Time", "Duration Time"])
        for k, v in mergedCCT.items():
            tmp = [k]
            tmp.append(v)
            writer.writerow(tmp)

{47.0: [10, 877280, 877270], 5.0: [1390, 4460, 3070], 73.0: [8830, 751670, 742840], 127.0: [11050, 732580, 721530], 211.0: [11860, 732710, 720850], 231.0: [16540, 735860, 719320], 259.0: [14380, 825530, 811150], 274.0: [17100, 889580, 872480], 303.0: [20320, 1635430, 1615110], 389.0: [588920, 848590, 259670], 393.0: [590710, 806280, 215570], 409.0: [596560, 1635670, 1039110], 475.0: [664130, 1081090, 416960], 503.0: [664330, 1637650, 973320], 11.0: [3940, 733100, 729160], 176.0: [11410, 756230, 744820], 264.0: [15490, 720970, 705480], 300.0: [20060, 792360, 772300], 308.0: [605830, 1079920, 474090], 'AverageCCT': 706005.2631578947}


In [50]:
with open("P4_RECORD/CoflowSize_Estimation_Record"+"_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow_"+str(len(sample_input_queue))+"packets_2.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["timer", "real cid", "global size", "mapping size", "local sizes", "queue sizes"])
        for k, v in coflow_size_estimation_record.items():
            tmp = [k]
            for item in v:
                tmp.append(item)
            writer.writerow(tmp)

In [18]:
for switch in switches:
    with open("P4_RECORD/classify_result_"+str(len(sample_f_id_list))+"flows_"+str(len(sample_input_queue))+"packets_"+str(numOfSwitches)+"switch"+"_"+str(numofCoflows)+"coflow"+"_"+str(switches.index(switch))+".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Time", "Real cid", "Classified cid", "Score", "Queue Distribution", "Representive", "Percentage", "Queue Size", "Dominated"])
        for k, v in switch.classify_result_table.items():
            tmp = [k]
            tmp.extend(v)
            writer.writerow(tmp)

In [14]:
partial = []
for key in switches[0].classify_result_table.keys():
    if switches[0].classify_result_table[key][0] == True and switches[0].classify_result_table[key][4]<=0.2:
        partial.append(switches[0].classify_result_table[key])
for item in partial:
    print(item)

[True, 90.0, 17.0, 0.045297022350132464, 0.038461538461538464, {17.0: {17.0: 1, 56.0: 24, 90.0: 1}, 20.0: {20.0: 7, 56.0: 11}, 55.0: {55.0: 18, 56.0: 1}, 59.0: {59.0: 2, 56.0: 6}, 56.0: {56.0: 172}}]
[True, 250.0, 227.0, 0.10374513640999794, 0.2, {93.0: {93.0: 5}, 119.0: {119.0: 13}, 129.0: {129.0: 2, 212.0: 1}, 177.0: {177.0: 3}, 204.0: {204.0: 1}, 227.0: {227.0: 1, 250.0: 4}}]
[True, 270.0, 227.0, 0.058754554390907286, 0.16666666666666666, {93.0: {93.0: 5}, 119.0: {119.0: 13}, 129.0: {129.0: 2, 212.0: 1}, 177.0: {177.0: 3}, 204.0: {204.0: 1}, 227.0: {227.0: 1, 250.0: 4, 270.0: 1}, 265.0: {265.0: 1}}]
[True, 270.0, 227.0, 0.06908884520332019, 0.14285714285714285, {93.0: {93.0: 5}, 119.0: {119.0: 13}, 129.0: {129.0: 2, 212.0: 1}, 177.0: {177.0: 3}, 204.0: {204.0: 1}, 227.0: {227.0: 1, 250.0: 4, 270.0: 2}, 265.0: {265.0: 1}}]
[True, 274.0, 265.0, 0.11005231738090515, 0.2, {93.0: {93.0: 5}, 119.0: {119.0: 13}, 129.0: {129.0: 2, 212.0: 1}, 177.0: {177.0: 3}, 204.0: {204.0: 1}, 227.0: {227